<a href="https://colab.research.google.com/github/Precillieo/Financial-Model/blob/main/Final_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
!pip install lightgbm
!pip install xgboost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
train= pd.read_excel('/content/drive/MyDrive/Mr Olowe/final dataset.xlsx')
val= pd.read_excel('/content/drive/MyDrive/Mr Olowe/Final Test Data.xlsx')
test= pd.read_excel('/content/drive/MyDrive/Mr Olowe/Final Test Data.xlsx')
train.head()

id  loan_amount  interest_due  ... lending_lenders  loans   status
0   9       288.88         72.22  ...               3     21  SETTLED
1  11       100.00         25.00  ...               3     21  SETTLED
2  13       100.00         25.00  ...               3     21  SETTLED
3  29      1000.00        250.00  ...               1      1  SETTLED
4  37      1000.00        250.00  ...               1      1  SETTLED

[5 rows x 40 columns]

In [3]:
print(val.shape)
print(train.shape)

(2500, 40)
(2500, 40)


In [4]:
train.status.value_counts()

SETTLED     1511
PAST DUE     989
Name: status, dtype: int64

# Visualizing The Target Value


In [5]:
import seaborn as sns
sns.countplot(x= train['status'])

# Filling Missing Values

In [6]:
train.dtypes

id                                 int64
loan_amount                      float64
interest_due                     float64
tenor                             object
interest_rate                    float64
card_network                      object
card_expiry                      float64
bank                              object
phone_network                      int64
date_of_birth             datetime64[ns]
tier                              object
selfie_id_check                   object
gender                            object
marital_status                    object
no_of_dependent                   object
type_of_residence                 object
educational_attainment            object
employment_status                 object
sector_of_employment              object
monthly_net_income                object
work_start_date           datetime64[ns]
work_email_validated               int64
address                           object
requested_amount                 float64
purpose         

In [7]:
train.isnull().sum()

id                           0
loan_amount                  0
interest_due                 0
tenor                        0
interest_rate               23
card_network                 0
card_expiry                 69
bank                        69
phone_network                0
date_of_birth                0
tier                         0
selfie_id_check              0
gender                    1940
marital_status               0
no_of_dependent              0
type_of_residence         1940
educational_attainment       0
employment_status            0
sector_of_employment         0
monthly_net_income           0
work_start_date              1
work_email_validated         0
address                   1940
requested_amount             0
purpose                      0
proposed_payday              0
credit_score                 0
location                     0
failed_requests              0
passed_requests              0
all_requests                 0
logins                       0
phone_nu

In [8]:
train.gender.value_counts(dropna=False)

NaN       1940
Male       409
Female     151
Name: gender, dtype: int64

In [9]:
train[~train.gender.isna()].status.value_counts()

PAST DUE    308
SETTLED     252
Name: status, dtype: int64

In [10]:
train.drop(['id','gender', 'type_of_residence', 'address'], 1, inplace=True)
train.columns

Index(['loan_amount', 'interest_due', 'tenor', 'interest_rate', 'card_network',
       'card_expiry', 'bank', 'phone_network', 'date_of_birth', 'tier',
       'selfie_id_check', 'marital_status', 'no_of_dependent',
       'educational_attainment', 'employment_status', 'sector_of_employment',
       'monthly_net_income', 'work_start_date', 'work_email_validated',
       'requested_amount', 'purpose', 'proposed_payday', 'credit_score',
       'location', 'failed_requests', 'passed_requests', 'all_requests',
       'logins', 'phone_numbers', 'emails', 'lenders', 'first_account',
       'last_account', 'lending_lenders', 'loans', 'status'],
      dtype='object')

In [11]:
train.interest_rate.value_counts()

25.20    279
35.49    109
15.00    104
19.99     56
11.33     47
        ... 
7.94       1
15.59      1
7.90       1
17.12      1
17.35      1
Name: interest_rate, Length: 371, dtype: int64

In [12]:
train.loc[train.interest_rate.isna()]

loan_amount  interest_due    tenor  ...  lending_lenders loans   status
28         3000.0         750.0  21 days  ...                3    10  SETTLED
29         3000.0         810.0  14 days  ...                3    30  SETTLED
31         2500.0         125.0   1 days  ...                3     6  SETTLED
32         2500.0         125.0   1 days  ...                2     3  SETTLED
33         2500.0         125.0   1 days  ...                3    10  SETTLED
34         2500.0         125.0   1 days  ...                2     3  SETTLED
35         2500.0         125.0   1 days  ...                3    30  SETTLED
36         2500.0         125.0   1 days  ...                3    10  SETTLED
40         2500.0         675.0  14 days  ...                1     2  SETTLED
41         2500.0         125.0   1 days  ...                1     2  SETTLED
42         2500.0         125.0   1 days  ...                1     1  SETTLED
45         2500.0         125.0   1 days  ...                3    30  SETTLED
46         2500.0         375.0   7 days  ...                3     8  SETTLED
1836       5000.0         750.0   7 days  ...                1     1  SETTLED
1837       5000.0         750.0   7 days  ...                1     1  SETTLED
1838       2500.0         125.0   1 days  ...                3    10  SETTLED
1839       2500.0         125.0   1 days  ...                3    30  SETTLED
1840       2500.0         125.0   1 days  ...                3    21  SETTLED
1841       2500.0         125.0   1 days  ...                3     6  SETTLED
1848       2500.0         125.0   1 days  ...                1     1  SETTLED
1851       3000.0         150.0   1 days  ...                4    23  SETTLED
1852       2500.0         125.0   1 days  ...                3    30  SETTLED
1853       2500.0         125.0   1 days  ...                3    30  SETTLED

[23 rows x 36 columns]

Every single one of them is ```settled```



The formula for interest due is:

***
$InterestDue = \frac{InterestRate * LoanAmount}{100}$
***
We could make `interest_rate` the subject of the formula:

*** 
$InterestRate = \frac{InterestDue * 100}{LoanAmount}$

In [13]:
train.loc[train.interest_rate.isna(), 'interest_rate'] = (train.interest_due * 100) / train.loan_amount

train.loc[train.interest_rate.isna()]

Empty DataFrame
Columns: [loan_amount, interest_due, tenor, interest_rate, card_network, card_expiry, bank, phone_network, date_of_birth, tier, selfie_id_check, marital_status, no_of_dependent, educational_attainment, employment_status, sector_of_employment, monthly_net_income, work_start_date, work_email_validated, requested_amount, purpose, proposed_payday, credit_score, location, failed_requests, passed_requests, all_requests, logins, phone_numbers, emails, lenders, first_account, last_account, lending_lenders, loans, status]
Index: []

# Card Expiry Column

In [14]:
train.card_expiry.value_counts()

22023.0     152
32023.0     139
82023.0     130
92023.0     125
72023.0     112
12023.0      92
122022.0     89
102023.0     85
92022.0      83
22022.0      82
82022.0      77
102022.0     75
12022.0      70
72022.0      67
62022.0      66
112022.0     65
122021.0     63
52023.0      60
42022.0      59
62023.0      59
32022.0      55
82021.0      51
52022.0      48
112021.0     48
112023.0     45
102021.0     41
92021.0      40
52021.0      33
72021.0      32
122023.0     29
42021.0      27
62021.0      26
42023.0      26
82024.0      23
102024.0     20
52024.0      19
32021.0      17
22024.0      14
72024.0      13
92024.0      13
12021.0      12
62024.0      11
12024.0      10
32024.0      10
42024.0       8
102020.0      3
22021.0       3
112024.0      2
122024.0      1
112020.0      1
Name: card_expiry, dtype: int64

In [15]:
train[train.card_expiry.isna()]

loan_amount  interest_due    tenor  ...  lending_lenders loans   status
0          288.88         72.22  15 days  ...                3    21  SETTLED
1          100.00         25.00  15 days  ...                3    21  SETTLED
2          100.00         25.00  15 days  ...                3    21  SETTLED
6         1234.56        308.64  15 days  ...                4    23  SETTLED
7         3000.00        750.00  15 days  ...                1     2  SETTLED
...           ...           ...      ...  ...              ...   ...      ...
2264      3750.00       1000.00  30 days  ...                1     2  SETTLED
2324      5000.00       1000.00  4 weeks  ...                1     3  SETTLED
2374      5000.00       1000.00  30 days  ...                1     3  SETTLED
2379      3600.00        907.20  14 days  ...                2     4  SETTLED
2497      5000.00       1000.00  4 weeks  ...                1     3  SETTLED

[69 rows x 36 columns]

- `card_expiry` and `bank` seems to have missing values in the same places. 
- Where there's a missing bank, there's a missing card expiry date. 
- All the cards are VISA cards. 
- Most of them are SETTLED, and most of them are located in Lagos.

In [16]:
train[train.card_network == 'Visa']['bank'].value_counts()

Access Bank                 249
United Bank for Africa       39
Access Bank (Diamond)        27
Polaris Bank                 14
Fidelity Bank                 7
First Bank of Nigeria         5
Ecobank Nigeria               5
First City Monument Bank      3
Zenith Bank                   1
Name: bank, dtype: int64

In [17]:
train.bank.value_counts()

Guaranty Trust Bank         653
Access Bank                 368
First Bank of Nigeria       306
Zenith Bank                 287
United Bank for Africa      260
Union Bank of Nigeria        83
Ecobank Nigeria              69
First City Monument Bank     67
Sterling Bank                65
Stanbic IBTC Bank            59
Fidelity Bank                59
Polaris Bank                 52
Access Bank (Diamond)        27
Wema Bank                    26
Keystone Bank                26
Unity Bank                   14
Jaiz Bank                     4
Heritage Bank                 3
SunTrust Bank                 2
Providus Bank                 1
Name: bank, dtype: int64

`Access bank` is the one with the highest number of VISA cards. We could fill the missing `bank` values with `Access Bank`.

In [18]:
train.loc[train.bank.isna(), 'bank'] = 'Access Bank'

#df[df.card_expiry.isna()]

# Cleaning Card Expiry Column

In [19]:
train[train.bank == 'Access Bank'].card_expiry.value_counts()

32023.0     55
22023.0     42
82023.0     38
12023.0     31
122022.0    21
52023.0     16
72023.0     14
102022.0    14
92022.0     13
122021.0    12
102023.0    11
112021.0    11
62023.0      9
92023.0      9
22022.0      9
82022.0      7
32021.0      6
12022.0      5
72021.0      5
112022.0     5
112023.0     4
42022.0      4
102021.0     4
72022.0      3
32022.0      3
62022.0      3
42021.0      3
52021.0      2
42023.0      2
62021.0      2
82021.0      2
92021.0      2
52022.0      1
Name: card_expiry, dtype: int64

In [20]:
train.loc[train.card_expiry.isna(), 'card_expiry'] = 32023.0

We could extract out the year and month of the card_expiry.

In [21]:
train['card_expiry_month'] = train.card_expiry.map(lambda x: str(int(x))[:-4]).astype(int)
train['card_expiry_year'] = train.card_expiry.map(lambda x: str(int(x))[-4:]).astype(int)
train.drop('card_expiry', 1, inplace=True)

train.head()

loan_amount  interest_due  ... card_expiry_month  card_expiry_year
0       288.88         72.22  ...                 3              2023
1       100.00         25.00  ...                 3              2023
2       100.00         25.00  ...                 3              2023
3      1000.00        250.00  ...                 7              2021
4      1000.00        250.00  ...                10              2021

[5 rows x 37 columns]

## Working With Date Related Columns (Date Of Birth & Work Start Date)

In [22]:
date_column= ['date_of_birth', 'work_start_date']

def extract_date(train,cols,):
    for x in cols:
        train[x +'_year'] = train[x].dt.year
#         df[x +'_day'] = df[x].dt.day
#         df[x +'_month'] = df[x].dt.month
#         df[x +'_quarter'] = df[x].dt.quarter
    train.drop(columns=date_column,axis=1,inplace=True)

In [23]:
extract_date(train,date_column)

# Another Date Columns(First Account & Last Account)

In [24]:
date_colu= ['first_account', 'last_account']

def extract_date(train,col,):
    for x in col:
        train[x +'_year'] = train[x].dt.year
        train[x +'_month'] = train[x].dt.month
#         df[x +'_day'] = df[x].dt.day
#         df[x +'_quarter'] = df[x].dt.quarter
    train.drop(columns=date_colu,axis=1,inplace=True)

In [25]:
extract_date(train, date_colu)

## Working On Date Columns (Tenor & Proposed Payday)

In [26]:
train['tenor'].value_counts()

15 days     573
30 days     527
1 months    500
14 days     281
21 days     138
4 weeks     134
3 weeks      80
7 days       74
1 days       42
20 days      41
28 days      28
25 days      20
10 days      17
16 days      16
29 days       8
22 days       5
18 days       4
17 days       4
27 days       3
26 days       2
24 days       1
19 days       1
12 days       1
Name: tenor, dtype: int64

In [27]:
train['tenor'] = train['tenor'].replace(['4 weeks', '3 weeks', '1 months'], ['28 days', '21 days', '30 days'])
train['tenor'] = train.tenor.map(lambda x: x.split(' ')[0]).astype(int)

train.tenor.value_counts()

30    1027
15     573
14     281
21     218
28     162
7       74
1       42
20      41
25      20
10      17
16      16
29       8
22       5
17       4
18       4
27       3
26       2
24       1
19       1
12       1
Name: tenor, dtype: int64

In [28]:
train.proposed_payday.value_counts()

15 days     895
30 days     719
1 months    504
4 weeks     134
3 weeks      80
20 days      41
21 days      28
28 days      28
25 days      20
16 days      16
29 days       8
22 days       5
2 months      5
18 days       4
17 days       4
27 days       3
26 days       2
24 days       1
4 months      1
19 days       1
12 days       1
Name: proposed_payday, dtype: int64

In [29]:
train.proposed_payday = train.proposed_payday.replace(['4 weeks', '3 weeks', '1 months', '2 months', '4 months'], ['28 days', '21 days', '30 days','60 days', '120 days' ])
train.proposed_payday = train.proposed_payday.map(lambda x: x.split(' ')[0]).astype(int)

train.proposed_payday.value_counts()

30     1223
15      895
28      162
21      108
20       41
25       20
16       16
29        8
60        5
22        5
17        4
18        4
27        3
26        2
120       1
24        1
19        1
12        1
Name: proposed_payday, dtype: int64

# Statistical Analysis

In [30]:
target_map = {
    'SETTLED': 2,
    'PAST DUE': 5}

train.status.replace(target_map, inplace = True)

In [31]:
corr = train.corr()
corr

loan_amount  ...  last_account_month
loan_amount              1.000000  ...            0.040457
interest_due             0.836875  ...            0.067319
tenor                    0.152191  ...            0.021093
interest_rate           -0.018697  ...            0.126059
phone_network           -0.020668  ...           -0.036913
work_email_validated    -0.052370  ...            0.014186
requested_amount         0.030243  ...           -0.000207
proposed_payday          0.092622  ...            0.013894
credit_score             0.094200  ...            0.013772
failed_requests         -0.018403  ...           -0.049890
passed_requests         -0.016829  ...           -0.095619
all_requests            -0.019266  ...           -0.060635
logins                   0.144402  ...           -0.052277
phone_numbers            0.000385  ...           -0.163067
emails                  -0.114253  ...           -0.206017
lenders                 -0.092431  ...           -0.266501
lending_lenders         -0.110632  ...           -0.140801
loans                   -0.117890  ...           -0.122529
status                  -0.049406  ...            0.120847
card_expiry_month        0.017494  ...            0.049713
card_expiry_year         0.016220  ...            0.002691
date_of_birth_year      -0.087913  ...           -0.038518
work_start_date_year    -0.043070  ...           -0.029182
first_account_year      -0.008261  ...           -0.181991
first_account_month      0.105343  ...            0.267597
last_account_year       -0.015842  ...           -0.917309
last_account_month       0.040457  ...            1.000000

[27 rows x 27 columns]

In [32]:
print(corr["status"].sort_values(ascending=True))

loans                  -0.326897
passed_requests        -0.169987
logins                 -0.127112
lending_lenders        -0.121185
all_requests           -0.106536
tenor                  -0.088103
failed_requests        -0.087282
proposed_payday        -0.074583
emails                 -0.064004
lenders                -0.063918
loan_amount            -0.049406
last_account_year      -0.038504
requested_amount       -0.020552
first_account_year     -0.006246
interest_due           -0.000731
credit_score            0.010877
phone_network           0.021119
work_start_date_year    0.022508
card_expiry_month       0.024648
phone_numbers           0.037131
card_expiry_year        0.048057
work_email_validated    0.058015
date_of_birth_year      0.103434
last_account_month      0.120847
interest_rate           0.180586
first_account_month     0.211310
status                  1.000000
Name: status, dtype: float64


In [33]:
train = pd.get_dummies(train)

In [34]:
train.columns

Index(['loan_amount', 'interest_due', 'tenor', 'interest_rate',
       'phone_network', 'work_email_validated', 'requested_amount',
       'proposed_payday', 'credit_score', 'failed_requests',
       ...
       'location_ogun', 'location_ondo', 'location_osun', 'location_oyo',
       'location_plateau', 'location_rivers', 'location_sokoto',
       'location_taraba', 'location_yobe', 'location_zamfara'],
      dtype='object', length=165)

In [35]:
#train['account_difference_month']= train['last_account_month']- train['first_account_month']

In [36]:
#train['account_difference_year']= train['last_account_year']- train['first_account_year']

In [37]:
missing_value=train.isnull().sum()
missing= missing_value[missing_value>0]

In [38]:
missing

work_start_date_year    1
dtype: int64

In [39]:
train.loc[train.work_start_date_year.isna(), 'work_start_date_year'] = 2018.0

## Modelling

### Train and Test Splitting

In [40]:
y= train['status']
X= train.drop('status', 1, inplace= True)

In [41]:
train_x, val_x, train_y, val_y = train_test_split(train.values, y.values, test_size=.2, random_state=99)

In [42]:
lgb= LGBMClassifier(learning_rate= 0.01, n_estimators= 100)
gb= GradientBoostingClassifier(learning_rate= 0.1, n_estimators= 200, max_depth= 2, min_samples_split=7)
xg= XGBClassifier(learning_rate=0.1, n_estimators= 200)

In [43]:
#vc = VotingClassifier(estimators=[('lgb', lgb), ('gb', gb), ('xg', xg)], weights=[2,1,1], voting='hard')

In [44]:
gb.fit(train_x, train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=2,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=7,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [45]:
pred_y = gb.predict(val_x)

In [46]:
print(classification_report(val_y, pred_y))

              precision    recall  f1-score   support

           2       0.90      0.87      0.88       299
           5       0.81      0.86      0.83       201

    accuracy                           0.86       500
   macro avg       0.86      0.86      0.86       500
weighted avg       0.86      0.86      0.86       500

